In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import re
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument('--headless')  # Optional: Run in headless mode
chrome_options.add_argument('--disable-gpu')  # Optional: Disable GPU for compatibility
chrome_options.add_argument('--no-sandbox')  # Optional: Required for some environments
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36")




In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains  # Add this line
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service

# Set ChromeDriver path and initialize Chrome options
chrome_options = Options()
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36")

service = Service('C:/chromedriver/chromedriver.exe')  # Update with your path
driver = webdriver.Chrome(service=service, options=chrome_options)

# Navigate to the website
url = 'https://www.imdb.com/title/tt15473010/releaseinfo/'
driver.get(url)

try:
    wait = WebDriverWait(driver, 20)  # Adjust timeout as needed

    # Wait for the page to load completely
    time.sleep(2)  # Adjust as necessary

    # Identify and handle the overlay
    overlay_locator = (By.CLASS_NAME, 'sc-kDvujY.kUNdqF')

    # Option 1: Wait for overlay to disappear
    try:
        wait.until(EC.invisibility_of_element_located(overlay_locator))
    except:
        print("Overlay is still present after wait.")

    # Option 2: Remove overlay via JavaScript
    driver.execute_script("""
        var overlay = document.querySelector('.sc-kDvujY.kUNdqF');
        if (overlay) {
            overlay.parentNode.removeChild(overlay);
            console.log('Overlay removed.');
        } else {
            console.log('Overlay not found.');
        }
    """)

    # Locate the button
    button_locator = (By.CLASS_NAME, 'ipc-see-more__button')
    button = wait.until(EC.element_to_be_clickable(button_locator))

    # Scroll the button into view
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", button)
    time.sleep(1)  # Allow time for scrolling

    # Verify button is visible and enabled
    if button.is_displayed() and button.is_enabled():
        try:
            # Try clicking with Selenium's click()
            button.click()
            print("Button clicked successfully with Selenium.")
        except Exception as e:
            print(f"Standard click failed: {e}")
            # Click using JavaScript
            driver.execute_script("arguments[0].click();", button)
            print("Button clicked successfully with JavaScript.")
    else:
        print("Button is not visible or enabled after scrolling.")

    # (Optional) Wait for new content to load
    # new_content_locator = (By.CLASS_NAME, 'new-content-class')
    # wait.until(EC.visibility_of_element_located(new_content_locator))

    # Proceed with scraping or other actions

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # Close the driver when done
    driver.quit()

Overlay is still present after wait.
Button clicked successfully with Selenium.


In [3]:

try:
    wait = WebDriverWait(driver, 20)  # Adjust timeout as needed

    # Handle the overlay if present (as before)
    overlay_locator = (By.CSS_SELECTOR, '.sc-kDvujY.kUNdqF')
    try:
        wait.until(EC.invisibility_of_element_located(overlay_locator))
    except:
        print("Overlay is still present after wait.")
        # Remove overlay via JavaScript
        driver.execute_script("""
            var overlay = document.querySelector('.sc-kDvujY.kUNdqF');
            if (overlay) {
                overlay.parentNode.removeChild(overlay);
                console.log('Overlay removed.');
            }
        """)

    # Locate and click the "19 more" button
    button_locator = (By.CLASS_NAME, 'ipc-see-more__button')
    button = wait.until(EC.element_to_be_clickable(button_locator))
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", button)
    time.sleep(1)

    if button.is_displayed() and button.is_enabled():
        try:
            button.click()
            print("Button clicked successfully with Selenium.")
        except Exception as e:
            print(f"Standard click failed: {e}")
            driver.execute_script("arguments[0].click();", button)
            print("Button clicked successfully with JavaScript.")
    else:
        print("Button is not visible or enabled after scrolling.")

    # Wait for the new content to load
    time.sleep(2)  # Adjust as necessary based on page load time

    # Locate all <li> elements with IDs starting with 'rel_'
    li_elements = driver.find_elements(By.XPATH, "//li[starts-with(@id, 'rel_')]")
    print(f"Found {len(li_elements)} <li> elements with IDs starting with 'rel_'.")

    # Iterate over the elements and extract information
    for li in li_elements:
        # Extract the ID
        li_id = li.get_attribute('id')
        print(f"\nProcessing element with ID: {li_id}")

        # Extract the link text and URL
        link = li.find_element(By.TAG_NAME, 'a')
        country = link.text
        country_url = link.get_attribute('href')
        print(f"Country: {country}")
        print(f"Country URL: {country_url}")

        # Extract the date and any subtext
        date_element = li.find_element(By.CSS_SELECTOR, 'span.ipc-metadata-list-item__list-content-item')
        date_text = date_element.text
        print(f"Date: {date_text}")

        # Check for subtext (e.g., "(internet)")
        subtext_elements = li.find_elements(By.CSS_SELECTOR, 'span.ipc-metadata-list-item__list-content-item--subText')
        if subtext_elements:
            subtext = subtext_elements[0].text
            print(f"Subtext: {subtext}")
        else:
            subtext = ''
            print("No subtext found.")

        # You can further extract other details or store the data as needed

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # Close the driver when done
    driver.quit()


Overlay is still present after wait.
An error occurred: HTTPConnectionPool(host='localhost', port=63601): Max retries exceeded with url: /session/e4acd9461ef120146843f8c71831ab2f/execute/sync (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000213B3938560>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))


In [4]:

try:
    wait = WebDriverWait(driver, 20)  # Adjust timeout as needed

    # Handle the overlay if present (as before)
    overlay_locator = (By.CSS_SELECTOR, '.sc-kDvujY.kUNdqF')
    try:
        wait.until(EC.invisibility_of_element_located(overlay_locator))
    except:
        # Remove overlay via JavaScript if still present
        driver.execute_script("""
            var overlay = document.querySelector('.sc-kDvujY.kUNdqF');
            if (overlay) {
                overlay.parentNode.removeChild(overlay);
            }
        """)

    # Locate and click the "19 more" button
    button_locator = (By.CLASS_NAME, 'ipc-see-more__button')
    button = wait.until(EC.element_to_be_clickable(button_locator))
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", button)
    time.sleep(1)

    if button.is_displayed() and button.is_enabled():
        try:
            button.click()
        except Exception:
            driver.execute_script("arguments[0].click();", button)
    else:
        print("Button is not visible or enabled after scrolling.")

    # Wait for the new content to load
    time.sleep(2)  # Adjust based on actual loading time

    # Locate all <li> elements with IDs starting with 'rel_'
    li_elements = driver.find_elements(By.XPATH, "//li[starts-with(@id, 'rel_')]")

    print(f"Found {len(li_elements)} <li> elements with IDs starting with 'rel_'.")

    # Extract data from each <li> element
    data = []
    for li in li_elements:
        # Extract the ID
        li_id = li.get_attribute('id')

        # Extract the <a> element
        link = li.find_element(By.TAG_NAME, 'a')

        # Extract the country from the 'aria-label' attribute
        country = link.get_attribute('aria-label')

        # Extract the date from the specified <span> element
        date_element = li.find_element(By.CSS_SELECTOR, 'span.ipc-metadata-list-item__list-content-item')
        date_text = date_element.text

        # Store the extracted data
        data.append({
            'id': li_id,
            'country': country,
            'date': date_text
        })

    # Print the extracted data
    for item in data:
        print(f"\nID: {item['id']}")
        print(f"Country: {item['country']}")
        print(f"Date: {item['date']}")

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # Close the driver when done
    driver.quit()


An error occurred: HTTPConnectionPool(host='localhost', port=63601): Max retries exceeded with url: /session/e4acd9461ef120146843f8c71831ab2f/execute/sync (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000213B3938B00>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))


In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options  # Added import
import time  # Added import

# Set ChromeDriver path and initialize Chrome options
chrome_options = Options()
chrome_options.add_argument("user-agent=Mozilla/5.0")  # Simplified user-agent

service = Service('C:/chromedriver/chromedriver.exe')  # Ensure this path is correct
driver = webdriver.Chrome(service=service, options=chrome_options)

# Navigate to the website
url = 'https://www.imdb.com/title/tt15473010/releaseinfo/'
driver.get(url)

try:
    wait = WebDriverWait(driver, 20)  # Adjust timeout as needed

    # Handle the overlay if present
    overlay_locator = (By.CSS_SELECTOR, '.sc-kDvujY.kUNdqF')
    try:
        wait.until(EC.invisibility_of_element_located(overlay_locator))
    except:
        # Remove overlay via JavaScript if still present
        driver.execute_script("""
            var overlay = document.querySelector('.sc-kDvujY.kUNdqF');
            if (overlay) {
                overlay.parentNode.removeChild(overlay);
            }
        """)

    # Locate and click the "19 more" button
    button_locator = (By.CLASS_NAME, 'ipc-see-more__button')
    button = wait.until(EC.element_to_be_clickable(button_locator))
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", button)
    time.sleep(1)

    if button.is_displayed() and button.is_enabled():
        try:
            button.click()
        except Exception as e:
            print(f"Standard click failed: {e}")
            driver.execute_script("arguments[0].click();", button)
    else:
        print("Button is not visible or enabled after scrolling.")

    # Wait for the new content to load
    time.sleep(2)  # Adjust based on actual loading time

    # Locate all <li> elements with IDs starting with 'rel_'
    li_elements = driver.find_elements(By.XPATH, "//li[starts-with(@id, 'rel_')]")

    print(f"Found {len(li_elements)} <li> elements with IDs starting with 'rel_'.")

    # Extract data from each <li> element
    data = []
    for li in li_elements:
        try:
            # Extract the ID
            li_id = li.get_attribute('id')

            # Extract the <a> element
            link = li.find_element(By.TAG_NAME, 'a')

            # Extract the country from the 'aria-label' attribute
            country = link.get_attribute('aria-label')

            # Extract the date from the specified <span> element
            date_element = li.find_element(By.CSS_SELECTOR, 'span.ipc-metadata-list-item__list-content-item')
            date_text = date_element.text

            # Store the extracted data
            data.append({
                'id': li_id,
                'country': country,
                'date': date_text
            })
        except Exception as e:
            print(f"Error processing li element with id {li.get_attribute('id')}: {e}")

    # Print the extracted data
    for item in data:
        print(f"\nID: {item['id']}")
        print(f"Country: {item['country']}")
        print(f"Date: {item['date']}")

except Exception as e:
    print(f"An error occurred: {e}")
    driver.save_screenshot('error_screenshot.png')  # Save screenshot for debugging
    raise  # Re-raise the exception to see the full traceback

finally:
    # Close the driver when done
    driver.quit()


Standard click failed: Message: element click intercepted: Element <button class="ipc-btn ipc-btn--single-padding ipc-btn--center-align-content ipc-btn--default-height ipc-btn--core-base ipc-btn--theme-base ipc-btn--button-radius ipc-btn--on-accent2 ipc-text-button ipc-see-more__button" tabindex="0" aria-disabled="false">...</button> is not clickable at point (72, 252). Other element would receive the click: <div class="sc-eDvSVe bZzAFe icb-html">...</div>
  (Session info: chrome=131.0.6778.86)
Stacktrace:
	GetHandleVerifier [0x00007FF69ADB3AB5+28005]
	(No symbol) [0x00007FF69AD183B0]
	(No symbol) [0x00007FF69ABB580A]
	(No symbol) [0x00007FF69AC0D6CE]
	(No symbol) [0x00007FF69AC0B16C]
	(No symbol) [0x00007FF69AC08628]
	(No symbol) [0x00007FF69AC0785D]
	(No symbol) [0x00007FF69ABF990E]
	(No symbol) [0x00007FF69AC2BA3A]
	(No symbol) [0x00007FF69ABF9246]
	(No symbol) [0x00007FF69AC2BC50]
	(No symbol) [0x00007FF69AC4B8B3]
	(No symbol) [0x00007FF69AC2B7E3]
	(No symbol) [0x00007FF69ABF75C8]


C:\Users\pangu\AppData\Local\Temp\ipykernel_32724\719127141.py:65: DeprecationWarning: using WebElement.get_attribute() has been deprecated. Please use get_dom_attribute() instead.
  li_id = li.get_attribute('id')
C:\Users\pangu\AppData\Local\Temp\ipykernel_32724\719127141.py:71: DeprecationWarning: using WebElement.get_attribute() has been deprecated. Please use get_dom_attribute() instead.
  country = link.get_attribute('aria-label')
C:\Users\pangu\AppData\Local\Temp\ipykernel_32724\719127141.py:84: DeprecationWarning: using WebElement.get_attribute() has been deprecated. Please use get_dom_attribute() instead.
  print(f"Error processing li element with id {li.get_attribute('id')}: {e}")


Error processing li element with id rel_aka_0: Message: no such element: Unable to locate element: {"method":"tag name","selector":"a"}
  (Session info: chrome=131.0.6778.86); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF69ADB3AB5+28005]
	(No symbol) [0x00007FF69AD183B0]
	(No symbol) [0x00007FF69ABB580A]
	(No symbol) [0x00007FF69AC05A3E]
	(No symbol) [0x00007FF69AC05D2C]
	(No symbol) [0x00007FF69ABF937C]
	(No symbol) [0x00007FF69AC2BA7F]
	(No symbol) [0x00007FF69ABF9246]
	(No symbol) [0x00007FF69AC2BC50]
	(No symbol) [0x00007FF69AC4B8B3]
	(No symbol) [0x00007FF69AC2B7E3]
	(No symbol) [0x00007FF69ABF75C8]
	(No symbol) [0x00007FF69ABF8731]
	GetHandleVerifier [0x00007FF69B0A643D+3118829]
	GetHandleVerifier [0x00007FF69B0F6C90+3448640]
	GetHandleVerifier [0x00007FF69B0ECF0D+3408317]
	GetHandleVerifier [0x00007FF69AE7A40B+841403]
	(No symbol) [0x00007F